## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
from __future__ import print_function
import tensorflow
import datetime
import keras
import pandas as pd
import os
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from PIL import Image
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split


Using TensorFlow backend.
/Users/huynh/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def get_image_paths(path, im):
    listing = os.listdir(path)
    count = 0
    for item in listing:

        if item == '.DS_Store':
            continue
        else:
            if '.jpg' in listing[0]:
                if count == 0:
                    im += [path + '/' + i for i in listing]
                    count += 1
            else:         
                get_image_paths(path + '/' + item, im)

# path to the images folder download from the website
dir_path = "/Users/huynh/Desktop/256/ut-zap50k-images"
# path to store new resized images
resize_path = '/Users/huynh/AnacondaProjects/256Project/resize_img'
im = []
get_image_paths(dir_path, im)

fp = open('combine_data.csv', 'w') 
for path in im:
    if path.split('/')[6] == 'Boots':
        fp.write(path + ',0\n')
    elif path.split('/')[6] == 'Sandals':
        fp.write(path + ',1\n')
    elif path.split('/')[6] == 'Shoes':
        fp.write(path + ',2\n')
    elif path.split('/')[6] == 'Slippers':
        fp.write(path + ',3\n')
fp.close()
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 300
num_classes = 4
epochs = 5

In [4]:
# set some more parameters
img_rows, img_cols = 50, 50
filters = 32
pool_size = 2
kernel_size = 5
img_channels = 1

In [5]:
imlist=[]
label = []
count = 0
with open('combine_data.csv') as f:
    for line in f:
        arr = line.split(',')
        imlist.append(arr[0])
        label.append(arr[1])
        count += 1

In [6]:
# create matrix to store all flattened images
c1 = 0

for path in imlist:
    temp = Image.open(path)
    temp = temp.resize((img_rows,img_cols))
    temp = temp.convert('L')
    temp.save(resize_path + '/' + label[c1][0:1] + '_' + str(c1) + '.jpg', "JPEG")
    c1 += 1



In [7]:
if not os.path.exists(resize_path):
    os.makedirs(resize_path)
    
imlist1 = os.listdir(resize_path)
immatrix = np.array([np.array(Image.open(resize_path + '/' + im2)).flatten() for im2 in imlist1])
immatrix.astype(float)
print(len(immatrix))
print(immatrix)


50025
[[255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 ..., 
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]]


In [8]:
label1 = np.ones((50025,),dtype = int)
c2 = 0
for item in imlist1:
    label1[c2] = item.split('_')[0]
    c2 += 1


data,Label = shuffle(immatrix,label1, random_state=2)
train_data = [data,Label]

(x, y) = (train_data[0], train_data[1])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)


In [9]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, x_train, x_test, y_train, y_test, num_classes):
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols,1)
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=1,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [10]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=(img_rows, img_cols,1)),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [11]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [12]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [13]:
# Let's take a look
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        832       
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 42, 32)        25632     
_________________________________________________________________
activation_2 (Activation)    (None, 42, 42, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 21, 21, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 14112)             0         
__________

In [14]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model, x_train, x_test, y_train, y_test, num_classes)

x_train shape: (40020, 50, 50, 1)
40020 train samples
10005 test samples
Train on 40020 samples, validate on 10005 samples
Epoch 1/1
40020/40020 [==============================] - 657s - loss: 0.5765 - acc: 0.8021 - val_loss: 0.3664 - val_acc: 0.8760
Training time: 0:10:57.912123
Test score: 0.366437373588
Test accuracy: 0.875962018996
